# Compute a Julia set with CUDA
**Author**: Stephan Hageboeck, CERN

In this notebook, we will write a simple GPU kernel to approximate a Julia (and a Fatou) set. For a complex number $z$ and a function $f(z)$,
- the Julia set is the set of numbers that is completely invariant under repeated application of $f(z)$,
- and the Fatou set is the complement of the Julia set.

See e.g. the [Wikipedia](https://en.wikipedia.org/wiki/Julia_set) for more details on Julia/Fatou sets.

For $f(z)$, we will use the following polynomial:
\begin{equation}
 f(z) = z^2 + c,
\end{equation}

where $c$ is a complex number that we can choose. If we apply this function repeatedly to $z$, and $z$ blows up quickly, it is part of the Fatou set. We will explore the behaviour of $f(z)$ on a complex plane of 1024x1024 pixels, so we will have to apply our function a few hundred times to more than one million numbers, which is an excellent task for a GPU. For each point on the complex plane, we will keep applying the function until either the number blows up, or the maximum number of iterations is reached. Depending on how fast the number blows up, we will colour the pixel in the complex plane.

This can look as follows for $f(z) = z^2 - 0.4 + 0.6i$:

<img src="juliaExample.png" width="400">

## CUDA progamming in a notebook?
It is indeed a bit ununsual to program CUDA in a Python notebook. We will use a few tricks to make it work. The notebook provides a uniform environment to all participants, such that we can access the SWAN GPUs without having to worry about the operating systems and installed software every participant brings. In order to compile the cuda programs, we will
- write a notebook cell into a `.cu` file
- compile it using the nvcc compiler
- invoke the application from the notebook
- and convert the image such that the notebook can display it.


## The Julia kernel
Below we have a draft cuda program to approximate the Julia/Fatou sets. We want to create a kernel that applies $f(z)$ to each point in the complex plane for `maxIter` times or until $z$ starts to diverge. The floating-point coordinates for a pixel at index `(i, j)` have already been computed, but you have to figure out the correct pixel indices and write the loop to repeatedly apply the function $f(z)$.

### Tasks
1. In the `main()` function, allocate memory for all pixels. The writePPM function expects an array of pixels in the form `{y0={x0 x1 x2 x3 ...}, y1={x0 x1 x2 x3 ...}, ... y_n }`, so allocate a one-dimensional array with enough memory for x*y pixels. There's already a section that checks for possible cuda errors, so allocate the memory just before. Don't forget to free the memory when you're done.
1. Launch the draft kernel from the main function. Check for possible errors.
1. Figure out a way to compute the pixel indices `i` and `j` from `threadIdx.x` and `blockIdx.x`. Find a kernel launch configuration that covers the entire image.
1. Implement the computation `z = z^2 + c`.
   - We will not use any external complex number classes for this, so square the complex number by hand.
   - Plug the computation in the loop. Check that it runs
     - for a maximum of `maxIter` times
     - or until z starts to diverge (`|z| >= maxMagnitude`).
   - Check that the iteration at which z diverged is recorded in the pixel array. If it didn't diverge (e.g. because the point is part of the Julia set), set the pixel to 0.
   - Note: We use 256 colours to colour the resulting image. We scale `k` into the range `[1, 256]` for best contrast, but it's not strictly necessary.
1. Check if you can generate a Julia image like in the example.


In [ ]:
! mkdir -p

In [ ]:
%%writefile tmp/julia.cu
#include "Timer.h"
#include "julia.h"

#include <iostream>
#include <cstdio>

// This function writes an image to disk
void writePPM(unsigned char const * pixels, size_t nx, size_t ny, const char * filename);

// Use this to change the floating-point precision in the kernel
using FPType = double;

// We use this to set up our image:
struct ImageDimensions {
  FPType xmin;
  FPType xmax;
  size_t nx;
  FPType ymin;
  FPType ymax;
  size_t ny;
};

__global__
void julia(const ImageDimensions dim, size_t maxIter, FPType maxMagnitude,
           unsigned char * image, FPType cReal, FPType cImag)
{
  // Compute the size of a pixel in x and y direction
  const FPType dx = (dim.xmax - dim.xmin) / dim.nx;
  const FPType dy = (dim.ymax - dim.ymin) / dim.ny;

  // Task 3: From threadIdx and blockIdx, compute the indices i and j
  // to address the pixels in x and y direction
  // ------------------------------------------------------------------
  const size_t i = 0;
  const size_t j = 0;

  if (i >= dim.nx || j >= dim.ny) return;

  // Compute the starting values for z based on the pixel location
  FPType zReal = dim.xmin + i * dx;
  FPType zImag = dim.ymin + j * dy;

  // Task 4: Compute Julia set
  // -----------------------------
  size_t k = 0;
  while (k < maxIter && (zReal*zReal + zImag*zImag) < maxMagnitude*maxMagnitude) {
    // Compute z^2 + c for complex numbers:


    ++k;
  }

  image[i + dim.nx*j] = k < maxIter ? 1 + (255 * k)/maxIter : 0;
}


int main(int argc, char * argv[]) {
  // Set up:
  constexpr double plotRange = 1.6;
  const FPType cReal = argc > 1 ? std::stod(argv[1]) : -0.4;
  const FPType cImag = argc > 2 ? std::stod(argv[2]) :  0.6;
  constexpr size_t sizeX = 1024;
  constexpr size_t sizeY = 1024;
  const ImageDimensions dim{-plotRange, plotRange, sizeX, -plotRange, plotRange, sizeY};

  // Task 1: Allocate memory
  // -----------------------
  unsigned char * pixels;


  if (const auto errorCode = cudaGetLastError(); errorCode != cudaSuccess) {
    std::cerr << "When allocating memory, encountered cuda error " << errorCode << " '"
              << cudaGetErrorName(errorCode)
              << "' with description:"
              << cudaGetErrorString(errorCode) << "\n";
    return 2;
  }

  /* call julia kernel to draw the Julia set into a buffer */
  {
    Timer kernelTimer{ "Compute Julia set" };

    // Task 2: Launch the kernel
    // -------------------------
    constexpr auto nThread = 1;
    constexpr auto nBlock = 1;

    julia<<<nBlock, nThread>>>(dim, 256, 2.f, pixels, cReal, cImag);

    if (const auto errorCode = cudaDeviceSynchronize(); errorCode != cudaSuccess) {
      std::cerr << "When submitting kernel, encountered cuda error '"
                << cudaGetErrorName(errorCode)
                << "' with description:"
                << cudaGetErrorString(errorCode) << "\n";
      return 3;
    }
  }

  // write GPU arrays to disk as PPM image
  writePPM(pixels, sizeX, sizeY, "julia.ppm");

  return 0;
}



## Compile, execute, display
To have consistent line numbers when compiling, we create an extra file that accounts for the empty line that's occupied by the `writefile` magic.
For the compilation step, we add `-g` to have debug symbols in the executable, `-std=c++17` for modern C++, and `-O2` to benefit from compiler optimisations on the host side.

In [ ]:
%%bash
sed '1s/^/\n/' < tmp/julia.cu > tmp/julia_extraLine.cu
nvcc -I source/ tmp/julia_extraLine.cu -std=c++17 -g -O2 -o tmp/julia

### Execute
You can pass the real and imaginary part for $c$ as arguments to the executable.

Try for example:
- `tmp/julia -0.4 0.6`
- `tmp/julia 0.285 -0.01`

IPython display doesn't natively support ppm images. Therefore, we use ImageMagick's `convert` to convert to png. A possible warning about a missing configure file can be ignored.

In [ ]:
%%bash
tmp/julia -0.4 0.6 && convert -size 256x256 julia.ppm julia.png

### Display the output
Note that what you see is mostly the Fatou set. We colour pixels based on how fast the function blew up. The Julia set is dark.

In [ ]:
from IPython.display import Image
Image(filename='julia.png')

## Details on the image file
If you are interested how we go from iteration number to the images, check the `writePPM` function in `julia.h`.
The ppm image format P3 is a simple but inefficient image format to dump the output of our kernel. In P3, we can simply write the RGB values of every pixel as ASCII numbers.
The files look as follows:
```
P3
width height
maxColorValue
RGB pixelValues (starting from top left pixel)
```
White spaces are ignored, so the writePPM function adds tabs to separate the pixels and line breaks after each line.

In [ ]:
%cat julia.ppm

# Bonus Tasks
## Floating-point precision
The draft kernel was using double precision for the complex numbers. Check if the usage of single-precision floating-point numbers give satisfying results, and check the impact on the kernel execution. How fast can we go?

## CPU equivalent
There is a naive CPU kernel called `juliaCPU` that can be used as a drop-in replacement for the GPU kernel. Check its speed. How much speed up against a single CPU thread can you reach with the GPU?

## Grid-strided loops for an image of arbitrary size
You might have written a kernel where `i = threadIdx.x`. Whereas this is sufficient for our problem size, the maximum number of threads per SM is 1024, so your kernel might not be able to deal with larger images. Remember that you can use the grid-strided loop to process an array of arbitray size. Try using a grid-strided loop on an image of 2048x2048 pixels. You can use a linearised index from 0 to 2048x2048, and compute i and j using modulus and integer division.

# Solution?
One possible solution can be found in [source/solution/julia.cu](source/solution/julia.cu)

In [ ]:
%load source/solution/julia.cu